# Section 2

### Akka HTTP client to query object detection

In [ ]:
import $ivy.`com.sksamuel.scrimage:scrimage-core_2.12:2.1.8`
import sys.process._
import javax.imageio.ImageIO
import java.io.File

In [ ]:
import $ivy.`io.circe:circe-core_2.12:0.10.1`
import $ivy.`io.circe:circe-generic_2.12:0.10.1`
import $ivy.`io.circe:circe-parser_2.12:0.10.1`
import _root_.io.circe.{Decoder, Encoder}

import io.circe.parser.decode

case class Detection(label: String, score: Float, x1: Double, y1: Double, x2: Double, y2: Double)

object Detection {
  import _root_.io.circe.generic.semiauto._

  implicit lazy val encoder: Encoder[Detection] = deriveEncoder[Detection]
  implicit lazy val decoder: Decoder[Detection] = deriveDecoder[Detection]
}

In [ ]:
val imageFile = "resources/baywatch.jpg"

In [ ]:
def detectObjects(s: String) = {
    val cmd = s"""curl -X POST -F img=@${s}  localhost:8080/detect"""
    println(cmd)
    cmd.!!
}

In [ ]:
detectObjects(imageFile)

In [ ]:
import java.io.ByteArrayInputStream
import javax.imageio.ImageIO
import java.awt.image.BufferedImage
import java.awt.{Color, Graphics2D}
import java.awt.geom.Ellipse2D
import java.awt.Font
import java.awt.BasicStroke
import java.awt.geom.AffineTransform
import java.awt.RenderingHints
import scala.math.{ sqrt, abs, min, max, exp, Pi }

In [ ]:
def customDrawBoundingBoxes(nameImage: String, threshold: Float) {
    val img = ImageIO.read(new File(nameImage))
    val graphics = img.createGraphics
    val w = img.getWidth
    val h = img.getHeight
    val font = new Font("Serif", Font.PLAIN, 30)
    val affineTransform = new AffineTransform()
    val oldStroke = graphics.getStroke()
    affineTransform.rotate(-Pi/2)
    val rotatedFont = font.deriveFont(affineTransform)
    graphics.setFont(font)
    graphics.setColor(Color.red)
    
    val response = s"""curl -X POST -F img=@${nameImage}  localhost:8080/detect"""!!
    
    val detections = decode[Seq[Detection]](response) match {
        case Right(detections) => detections
        case Left(_) => List[Detection]()
    }
    println(detections)
    detections.filter(_.score > threshold).foreach { b =>
        graphics.setColor(Color.red)
        graphics.drawString((b.label).toString, b.x1.toInt, b.y1.toInt)
        graphics.setStroke(new BasicStroke(1.0f))
        graphics.drawRect(b.x1.toInt, b.y1.toInt, (b.x2 - b.x1).toInt, (b.y2 - b.y1).toInt) 
        graphics.setStroke(oldStroke)
    }
    graphics.dispose()
    lazy val baos = new java.io.ByteArrayOutputStream()
    ImageIO.write( img, "png", baos );
    baos.flush()
    lazy val imageInByte = baos.toByteArray()
    baos.close()
    Image(imageInByte).withFormat(Image.PNG).display 
}

In [ ]:
customDrawBoundingBoxes(imageFile, 0.9f)

In [ ]:
customDrawBoundingBoxes("resources/elephant.jpg", 0.9f)

In [ ]:
customDrawBoundingBoxes("resources/couvert.jpg", 0.9f)